In [1]:
import datetime
import numpy as np
import pandas as pd
import requests
import scipy
import scipy.sparse as sp
import json

import os
import pickle
import json

from bs4 import BeautifulSoup

from utils import weatherapi
from utils import isw_preprocessing
from utils import text_preprocessing

from sklearn import preprocessing


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dusty3ntity/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dusty3ntity/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dusty3ntity/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
MODELS_FOLDER = "data/models"
OUTPUT_FOLDER = "data/prediction"

if not os.path.exists(OUTPUT_FOLDER):
   os.makedirs(OUTPUT_FOLDER)

ISW_DATA_FILE = "data/isw/reports_processed_v2.csv"
WEATHER_EVENTS_DATA_FILE = "data/alarms/merged_events.csv"

tfidf_transformer_model = "tfidf_transformer"
tfidf_transformer_version = "v1"

count_vectorizer_model = "count_vectorizer"
count_vectorizer_version = "v1"

decision_tree_model = "DTC"
logistic_regression_model = "LR"
multi_layer_perceptron_model = "MLP"
random_forest_classifier_model = "RFC"
sgd_classifier_model = "SGD"

## 1. Load models & data

In [4]:
tfidf = pickle.load(open(f"data/isw/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"data/isw/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

label_encoder = pickle.load(open(f"data/isw/hour_conditions_label_encoder.pkl", "rb"))

DTC = pickle.load(open(f"{MODELS_FOLDER}/{decision_tree_model}.pkl", "rb"))
LR = pickle.load(open(f"{MODELS_FOLDER}/{logistic_regression_model}.pkl", "rb"))
MLP = pickle.load(open(f"{MODELS_FOLDER}/{multi_layer_perceptron_model}.pkl", "rb"))
RFC = pickle.load(open(f"{MODELS_FOLDER}/{random_forest_classifier_model}.pkl", "rb"))
SGD = pickle.load(open(f"{MODELS_FOLDER}/{sgd_classifier_model}.pkl", "rb"))

In [5]:
REGIONS_DICTIONARY_FILE = "data/data_before_lab_3/regions.csv"
df_regions = pd.read_csv(REGIONS_DICTIONARY_FILE)
df_regions.head(25)

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5
5,Житомирська,Житомир,Zhytomyr,Житомирщина,6
6,Закарпатська,Ужгород,Uzhgorod,Закарпаття,7
7,Запорізька,Запоріжжя,Zaporozhye,Запоріжжя,8
8,Івано-Франківська,Івано-Франківськ,Ivano-Frankivsk,Івано-Франківщина,9
9,Київська,Київ,Kyiv,Київщина,10


## 2. Setup prediction region

In [31]:
region = "Львівщина"
date = "2023-04-22"

city = df_regions[df_regions["region_alt"]==region]["center_city_en"].values[0]

## 3. Get weather

In [32]:
location = f"{city},Ukraine"

file_name = f"weather___{city.lower()}__{date}.json"

In [33]:
if not os.path.isfile(f"{OUTPUT_FOLDER}/{file_name}"):
	city_weather_json = weatherapi.get_hourly_weather_json(location, date)
	json_object = json.dumps(city_weather_json, indent=4)

	with open(f"{OUTPUT_FOLDER}/{file_name}", "w") as outfile:
		outfile.write(json_object)
else:
	print(f"Weather data for {region} and {date} is ready")

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Lviv,Ukraine/2023-04-22?unitGroup=metric


In [34]:
weather_file_path = f"{OUTPUT_FOLDER}/{file_name}"
weather_for_day_hourly = json.load(open(weather_file_path, "rb"))
weather_df = pd.DataFrame.from_dict(weather_for_day_hourly)

# Getting random weather from the dataset as the API is not ready yet
# all_weather_by_hour = pd.read_csv("data/data_before_lab_3/all_weather_by_hour.csv")
# weather_df = all_weather_by_hour.loc[(all_weather_by_hour['day_datetime'] == "2022-04-08") & (all_weather_by_hour['city_address'] == location)]

weather_df["day_datetime"] = pd.to_datetime(weather_df["day_datetime"])
weather_df = weather_df.fillna(-10)
weather_df["city"] = weather_df["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
weather_df["city"] = weather_df["city"].replace("Хмельницька область", "Хмельницький")

In [35]:
print(weather_df.shape)
weather_df.head(5)

(24, 68)


,hour_severerisk,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_severerisk,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_feelslikemax,day_feelslikemin,day_feelslike,day_dew,day_humidity,day_precip,day_precipprob,day_precipcover,day_preciptype,day_snow,day_snowdepth,day_windgust,day_windspeed,day_winddir,day_pressure,day_cloudcover,day_visibility,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunriseEpoch,day_sunset,day_sunsetEpoch,day_moonphase,day_conditions,day_description,day_icon,day_stations,day_source,hour_datetime,hour_datetimeEpoch,hour_temp,hour_feelslike,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_conditions,hour_icon,hour_stations,hour_source,city
0,10.0,49.8444,24.0254,"Львів, Україна","Lviv,Ukraine",Europe/Kiev,3.0,10.0,2023-04-22,1682110800,16.2,7.5,12.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,0.0,[rain],0.0,0.0,23.0,8.6,5.1,1015.3,55.3,22.0,187.7,16.3,7.0,06:18:14,1682133494,20:27:49,1682184469,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[remote],comb,00:00:00,1682110800,11.0,11.0,43.97,-0.8,0.0,0.0,0.0,0.0,-10,9.0,4.0,354.3,1015.0,24.1,91.6,0.0,-10.0,0.0,Overcast,cloudy,[remote],obs,Львів
1,10.0,49.8444,24.0254,"Львів, Україна","Lviv,Ukraine",Europe/Kiev,3.0,10.0,2023-04-22,1682110800,16.2,7.5,12.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,0.0,[rain],0.0,0.0,23.0,8.6,5.1,1015.3,55.3,22.0,187.7,16.3,7.0,06:18:14,1682133494,20:27:49,1682184469,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[remote],comb,01:00:00,1682114400,10.3,10.3,46.07,-0.8,0.0,0.0,0.0,0.0,-10,8.6,4.0,357.6,1015.0,24.1,68.0,0.0,-10.0,0.0,Partially cloudy,partly-cloudy-night,[remote],obs,Львів
2,10.0,49.8444,24.0254,"Львів, Україна","Lviv,Ukraine",Europe/Kiev,3.0,10.0,2023-04-22,1682110800,16.2,7.5,12.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,0.0,[rain],0.0,0.0,23.0,8.6,5.1,1015.3,55.3,22.0,187.7,16.3,7.0,06:18:14,1682133494,20:27:49,1682184469,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[remote],comb,02:00:00,1682118000,9.7,9.7,47.95,-0.8,0.0,0.0,0.0,0.0,-10,7.9,3.2,1.5,1015.0,24.1,64.0,0.0,-10.0,0.0,Partially cloudy,partly-cloudy-night,[remote],obs,Львів
3,10.0,49.8444,24.0254,"Львів, Україна","Lviv,Ukraine",Europe/Kiev,3.0,10.0,2023-04-22,1682110800,16.2,7.5,12.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,0.0,[rain],0.0,0.0,23.0,8.6,5.1,1015.3,55.3,22.0,187.7,16.3,7.0,06:18:14,1682133494,20:27:49,1682184469,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[remote],comb,03:00:00,1682121600,8.9,8.9,58.08,1.1,0.0,0.0,0.0,0.0,-10,4.0,2.2,357.9,1015.0,24.1,34.7,0.0,-10.0,0.0,Partially cloudy,partly-cloudy-night,[remote],obs,Львів
4,10.0,49.8444,24.0254,"Львів, Україна","Lviv,Ukraine",Europe/Kiev,3.0,10.0,2023-04-22,1682110800,16.2,7.5,12.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,0.0,[rain],0.0,0.0,23.0,8.6,5.1,1015.3,55.3,22.0,187.7,16.3,7.0,06:18:14,1682133494,20:27:49,1682184469,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,[remote],comb,04:00:00,1682125200,8.5,8.5,57.97,0.7,0.0,0.0,0.0,0.0,-10,4.3,2.2,357.4,1015.0,24.1,0.0,0.0,-10.0,0.0,Clear,clear-night,[remote],obs,Львів


In [36]:
weather_df_v2 = pd.merge(weather_df, df_regions, left_on="city", right_on="center_city_ua")

## 3. Get ISW report

In [37]:
isw_report_html_file = f"{OUTPUT_FOLDER}/isw___{date}.html"

In [38]:
if not os.path.isfile(isw_report_html_file):
	BASE_URL = "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment"
	months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
	date_arr = date.split("-")

	url = f"{BASE_URL}-{months[int(date_arr[1])-1]}-{int(date_arr[2])}-{int(date_arr[0])}"
	print(url)

	page = requests.get(url)
	isw_report_html_file = f"{OUTPUT_FOLDER}/isw___{date}.html"
	with open(isw_report_html_file, "wb+") as f:
		f.write(page.content)
else:
	print(f"ISW report for {date} is ready")

https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-april-22-2023


In [39]:
with open(isw_report_html_file, "r", encoding="utf8") as cfile:
	html = BeautifulSoup(cfile.read(), features="html.parser")
	content_html = str(html.body.find("div", attrs={"class": "field-type-text-with-summary"}))

	content_html = isw_preprocessing.preprocess_page_html(content_html)
	content_text_lemm = text_preprocessing.text_preprocess(content_html, "lemm")

In [40]:
word_count_vector = cv.transform([content_text_lemm])

In [41]:
word_count_vector

<1x6524 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [42]:
tfidf_vector = tfidf.transform(word_count_vector)

In [43]:
tfidf_vector

<1x6524 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

## 4. Data preparation

In [44]:
weather_exclude = [
	"day_feelslikemax",
	"day_feelslikemin",
	"day_sunriseEpoch",
	"day_sunsetEpoch",
	"day_description",
	"city_latitude",
	"city_longitude",
	"city_address",
	"city_timezone",
	"city_tzoffset",
	"day_feelslike",
	"day_precipprob",
	"day_snow",
	"day_snowdepth",
	"day_windgust",
	"day_windspeed",
	"day_winddir",
	"day_pressure",
	"day_cloudcover",
	"day_visibility",
	"day_severerisk",
	"day_conditions",
	"day_icon",
	"day_source",
	"day_preciptype",
	"day_stations",
	"hour_icon",
	"hour_source",
	"hour_stations",
	"hour_feelslike"
]

fields_to_exclude = [
	"city_resolvedAddress",
	"day_datetime",
	"day_datetimeEpoch",
	"hour_datetime",
	"hour_datetimeEpoch",
	"city",
	"region",
	"center_city_ua",
	"center_city_en"
]

tmp_fields_to_exclude = [
	"day_sunrise",
	"day_sunset",
	"hour_preciptype",
	"hour_conditions",
	"hour_solarenergy",
	"region_alt"
]

df_work_v2 = weather_df_v2.drop(weather_exclude, axis=1)
df_work_v2 = df_work_v2.drop(fields_to_exclude, axis=1)

In [45]:
df_work_v2["hour_conditions"] = df_work_v2["hour_conditions"].apply(lambda x: x.split(",")[0])
df_work_v2["hour_conditions_id"] = label_encoder.transform(df_work_v2["hour_conditions"])

In [46]:
df_work_v3 = df_work_v2.drop(tmp_fields_to_exclude, axis=1)

In [47]:
df_work_v3.head(10)

,hour_severerisk,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_moonphase,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_uvindex,region_id,hour_conditions_id
0,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,11.0,43.97,-0.8,0.0,0.0,0.0,0.0,9.0,4.0,354.3,1015.0,24.1,91.6,0.0,0.0,13,3
1,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,10.3,46.07,-0.8,0.0,0.0,0.0,0.0,8.6,4.0,357.6,1015.0,24.1,68.0,0.0,0.0,13,4
2,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,9.7,47.95,-0.8,0.0,0.0,0.0,0.0,7.9,3.2,1.5,1015.0,24.1,64.0,0.0,0.0,13,4
3,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,8.9,58.08,1.1,0.0,0.0,0.0,0.0,4.0,2.2,357.9,1015.0,24.1,34.7,0.0,0.0,13,4
4,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,8.5,57.97,0.7,0.0,0.0,0.0,0.0,4.3,2.2,357.4,1015.0,24.1,0.0,0.0,0.0,13,0
5,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,8.0,59.97,0.7,0.0,0.0,0.0,0.0,3.6,2.5,357.4,1015.0,24.1,3.5,0.0,0.0,13,0
6,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,7.6,61.19,0.6,0.0,0.0,0.0,0.0,3.6,2.5,358.9,1015.0,24.1,33.7,0.0,0.0,13,4
7,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,7.5,61.61,0.6,0.0,0.0,0.0,0.0,3.6,1.8,308.2,1015.0,24.1,59.6,11.0,0.0,13,4
8,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,8.0,61.73,1.1,0.0,0.0,0.0,0.0,5.0,2.9,284.0,1016.0,24.1,55.3,116.0,1.0,13,4
9,10.0,16.2,7.5,12.0,1.3,49.0,0.0,0.0,187.7,16.3,7.0,0.07,9.1,60.25,1.8,0.0,0.0,0.0,0.0,7.6,2.5,273.7,1016.0,24.1,48.6,267.0,3.0,13,4


In [48]:
tfidf_matrix = tfidf_vector

for i in range(0, 23):
	tfidf_matrix = sp.vstack((tfidf_matrix, tfidf_vector), format="csr")

In [49]:
tfidf_vector

<1x6524 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [50]:
tfidf_matrix

<24x6524 sparse matrix of type '<class 'numpy.float64'>'
	with 144 stored elements in Compressed Sparse Row format>

In [51]:
df_work_v4_csr = sp.csr_matrix(df_work_v3.values)
df_work_v4_csr

<24x29 sparse matrix of type '<class 'numpy.float64'>'
	with 524 stored elements in Compressed Sparse Row format>

In [52]:
df_all_features = sp.hstack((df_work_v4_csr, tfidf_matrix), format="csr")

In [53]:
hours_alarm_schedule = DTC.predict(df_all_features)
hours_alarm_schedule

array([1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0])

In [56]:
schedule = []
for i in range(0, 24):
	schedule.append({ f"{i}:00": "false" if hours_alarm_schedule[i] == 0 else "true" })

print(f"Alarms in {location} on {date}:")
schedule

Alarms in Lviv,Ukraine on 2023-04-22:


[{'0:00': 'true'},
 {'1:00': 'true'},
 {'2:00': 'true'},
 {'3:00': 'true'},
 {'4:00': 'false'},
 {'5:00': 'false'},
 {'6:00': 'true'},
 {'7:00': 'true'},
 {'8:00': 'true'},
 {'9:00': 'true'},
 {'10:00': 'true'},
 {'11:00': 'true'},
 {'12:00': 'true'},
 {'13:00': 'true'},
 {'14:00': 'true'},
 {'15:00': 'true'},
 {'16:00': 'true'},
 {'17:00': 'true'},
 {'18:00': 'true'},
 {'19:00': 'true'},
 {'20:00': 'true'},
 {'21:00': 'false'},
 {'22:00': 'false'},
 {'23:00': 'false'}]